#### Daily Data Pull

In [17]:
# imports
import pandas as pd
import numpy as np
import requests
import time
import datetime
from nltk.tokenize import RegexpTokenizer

In [8]:
# set display options 
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

In [2]:
# define function to return minimum postedDate as string
def get_min_post_date(filename):
    # read in file of downloaded contract opportunities
    df = pd.read_csv(filename)
    # get min posted date
    min_date = pd.to_datetime(df['postedDate']).min()
    return min_date.strftime('%m/%d/%Y')

In [3]:
# define function to return maximum postedDate as string
def get_max_post_date(filename):
    # read in file of downloaded contract opportunities
    df = pd.read_csv(filename)
    # get min posted date
    max_date = pd.to_datetime(df['postedDate']).max()
    return max_date.strftime('%m/%d/%Y')

In [4]:
# define function to return today's date as string
def today():
    now = datetime.datetime.now()
    return now.strftime('%m/%d/%Y')

In [7]:
# pull contract information from api.sam.gov

# set base url
url = 'https://api.sam.gov/prod/opportunities/v1/search'

# create empty list to store results
result = []
# initialize counter
count = 0
# downloaded contract opportunities - file name
# file = './data/combined.csv'

# set postedTo date to today's date by calling today() function
postedTo = today()

# set posted from date
postedFrom = get_max_post_date('./data/combined.csv')

# for loop to pull contracts
for i in range(8):
    count += 1
    
    # do a get request
    req = requests.get(url,
                      params={
                          'api_key': '',
                          'postedFrom': postedFrom,
                          'postedTo': postedTo,
                          'limit': 1000,
                          'offset': (count-1) * 1000
                      })
    
    # add response to result list
    result.append(req)
    
    now = datetime.datetime.now()
    print('Time:', now.strftime("%Y-%m-%d %H:%M:%S"))
    time.sleep(5)
    
    
    
# source for datetime - https://www.w3resource.com/python-exercises/python-basic-exercise-3.php

Time: 2020-06-07 23:43:45
Time: 2020-06-07 23:44:12
Time: 2020-06-07 23:44:37
Time: 2020-06-07 23:45:02
Time: 2020-06-07 23:45:23
Time: 2020-06-07 23:45:31
Time: 2020-06-07 23:45:38
Time: 2020-06-07 23:45:44


In [8]:
# unpack list of json objects from response data
ops = []
for item in result:
    print(item.headers)
    ops.append(item.json())

{'Age': '18', 'Content-Type': 'application/hal+json', 'Date': 'Mon, 08 Jun 2020 03:43:45 GMT', 'Server': 'openresty', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': 'http/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Cache': 'MISS', 'X-Forwarded-For': '74.96.156.35, 10.177.16.72, 10.177.52.196, 10.177.52.196', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Set-Cookie': 'citrix_ns_id=JhG0yrlAzcpsZb20kl06r0ol5xU0001; Domain=.sam.gov; Path=/; Secure; HttpOnly', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked'}
{'Age': '22', 'Content-Type': 'application/hal+json', 'Date': 'Mon, 08 Jun 2020 03:44:12 GMT', 'Server': 'openresty', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': 'http/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Cache': 'MISS', 'X-Forwarded-For': '74.96.156.35, 10.177.16.72, 10.177.55.102

In [9]:
# parse json objects
ls_data = []
for i in range(len(ops)):
    print(ops[i].keys())
    df = pd.DataFrame(ops[i]['opportunitiesData'])
    ls_data.append(df)
    data = pd.concat(ls_data)

dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])


In [10]:
# check how many new data points were downloaded
data.shape

(4738, 27)

In [11]:
data.tail(10)

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,...,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
728,026dc1a7056e443f89e97c6fbaf134a3,Cabling & Installation,N6893620Q0216,DEPT OF DEFENSE,DEPT OF THE NAVY,NAVAL AIR WARFARE CENTER,2020-06-03,Presolicitation,Presolicitation,autocustom,...,None,"[{'fax': '', 'type': 'primary', 'email': 'yvon...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '93555-6018', 'city': 'CHINA LAKE'...",None,None,https://beta.sam.gov/opp/026dc1a7056e443f89e97...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",None
729,02677037d39e46cfa3c0c847df58daff,41--M1010 Freezer,165THINBRIGADE00003,DEPT OF DEFENSE,DEPT OF THE ARMY,W6QM MICC-FT DRUM,2020-06-03,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,...,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'ma...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '13602-5220', 'city': 'FORT DRUM',...","{'streetAddress': 'Forty Jackson, SC', 'zip': ...",None,https://beta.sam.gov/opp/02677037d39e46cfa3c0c...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",None
730,01b967aceff8423a9e585b8c86f84a43,"66--GYROSCOPE,RATE",SPE4A620T21L5,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DLA AVIATION,2020-06-03,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,...,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'Di...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '23297', 'city': 'RICHMOND', 'coun...",{},None,https://beta.sam.gov/opp/01b967aceff8423a9e585...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",None
731,015f68396c0c46f581ed8eb9c74e7031,Assault Breacher Vehicle Remote Control System...,PANDTA-20-P-0000-008592,DEPT OF DEFENSE,DEPT OF THE ARMY,W4GG HQ US ARMY TACOM,2020-06-03,Sources Sought,Sources Sought,auto15,...,None,"[{'fax': '', 'type': 'primary', 'email': 'mich...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '48397-5000', 'city': 'WARREN', 'c...",None,None,https://beta.sam.gov/opp/015f68396c0c46f581ed8...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",[https://beta.sam.gov/api/prod/opps/v3/opportu...
732,00b27f9d17aa4d43873df0f3732f522f,P200 - Sole Source No-Cost Disposal Service,N0018920RZ085,DEPT OF DEFENSE,DEPT OF THE NAVY,NAVSUP FLT LOG CTR NORFOLK,2020-06-03,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,...,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'ja...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '23511-3392', 'city': 'NORFOLK', '...",{},None,https://beta.sam.gov/opp/00b27f9d17aa4d43873df...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",None
733,006816da72d442aeac1bc7efe0afe201,Preventative Maintenance Emergency Repair of E...,FA481420P0042,DEPT OF DEFENSE,DEPT OF THE AIR FORCE,FA4814 6 CONS PK,2020-06-03,Justification,Justification,auto30,...,"{'date': '2020-05-22', 'number': 'FA481420P0042'}","[{'fax': '', 'type': 'primary', 'email': 'tric...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '33621-5119', 'city': 'TAMPA', 'co...","{'city': {'code': '71000', 'name': 'Tampa'}, '...",None,https://beta.sam.gov/opp/006816da72d442aeac1bc...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",[https://beta.sam.gov/api/prod/opps/v3/opportu...
734,00624941e33545db91e5d132a10f7c8c,"61--BATTERY,STORAGE",SPE7L720T4198,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DLA LAND AND MARITIME,2020-06-03,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,...,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'Di...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '43218-3990', 'city': 'COLUMBUS', ...",{},None,https://beta.sam.gov/opp/00624941e33545db91e5d...

In [12]:
# check if there are duplicate documents in the newly pulled data
data.duplicated('noticeId').sum()

0

In [15]:
# if duplicates found, drop those
# data.drop_duplicates('noticeId', inplace=True)

In [13]:
# save data into a new file
data.to_csv('./data/6_07_pull.csv', index=False)

**Note:** Because in Notebook 008 the combined.csv dataframe gets two additional columns -  'cleaned_titles' and 'date_posted', before concatenating the new pull with the combined.csv, we need a function that transforms the newly acquired data into the same shape.

In [18]:
# define function to tokenize a column 
# modified to return title cased titles

def tokenizer_function(column):
    """
    Takes in a text column
        tokenizes the text in each row
        using pattern [[a-zA-Z]\w+]
        which matches every lowercase and upperase character between a-z that are word characters
    Returns list of strings
    """
    
    # instantiate empty list of tokenized text
    texts = []
    
    # define tokenizer pattern
    pattern = '[a-zA-Z]\w+'
    # instantiate tokenizer
    tokenizer = RegexpTokenizer(pattern=pattern)
    
    # create for loop to tokenize each row and add the list of tokens to texts
    for text in column:
        tokens = tokenizer.tokenize(text)
        
        # transform tokens into lower case strings
        tokens = [token.title() for token in tokens]
        texts.append(' '.join(tokens))
    return texts

In [19]:
def transform_new_pull(new_data):
    """function
       accepts new_data
       adds a tokenized title column to new_data
       adds a column of posted date in datetime format to new_data
       returns transformed new_data as dataframe"""
    # create tokenized title column
    new_data['cleaned_titles'] = tokenizer_function(new_data['title'])
    
    # create date-posted column in datetime format
    new_data['date_posted'] = pd.to_datetime(new_data['postedDate'])
    
    return new_data

In [21]:
# transform new dataframe
data = transform_new_pull(data)

In [22]:
# save transformed dataframe into a csv file
data.to_csv('./data/6_07_pull_transformed.csv', index=False)

In [23]:
# read in both the combined dataframe and the new-tranformed dataframe
df1 = pd.read_csv('./data/combined.csv')
df2 = pd.read_csv('./data/6_07_pull_transformed.csv')

In [25]:
# concatenate the new dataframe with the combined dataframe
df = pd.concat([df1, df2])

In [26]:
# check new shape
df.shape

(55625, 29)

In [27]:
# overwrite the combined csv file with the newly combined dataframe
df.to_csv('./data/combined.csv', index=False)